# Water abstraction - Drawdown prediction with the Theis solution
This notebook demonstrate the application of the Theis solution to predict the drawdown from pumping in a confined aquifer.

## Introduction

### General situation
We consider a confined aquifer with constant transmissivity. If a well is pumping water out of the aquifer, radial flow towards the well is induced. The following figure illustrates this situation.

<img src="FIGS\Flow_well_confined.png" width="400">

*Fig: Radial flow in a confined aquifer (Fetter, 1994)*

The calculate the hydraulic situation, the following simplified flow equation can be used. This equation accounts for 1D radial transient flow towards a fully penetrating well within a confined aquifer without further sinks and sources:

$$ \frac{\partial^2 h}{\partial r^2}+\frac{1}{r}\frac{\partial h}{\partial r}=\frac{S}{T}\frac{\partial h}{\partial t} $$

### Solution by Theis
Charles V. Theis presented a solution for this by deriving

$$ s(r,t)=\frac{Q}{4\pi T}W(u) $$

with the well function

$$ W(u) = \int_{u }^{+\infty} \frac{e^{-\tilde u}}{\tilde u}d\tilde u $$

and the dimensionless variable 

$$ u = \frac{Sr^2}{4Tt} $$

This equations are not easy to solve. Historically, values for the well function were provided by tables or as so called type-curve. The type-curve matching with experimental data for pumping test analysis can be considered as one of the basic hydrogeological methods.

However, modern computer provide an easier and more convinient way to solve the 1D radial flow equation based on the Theis approach. Subsequently, the Theis equation is solved with Python routines.

In [7]:
# (First, the necessary Python functions are initiated.)

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
from ipywidgets import *

# (Here the necessary functions like the well function $W(u)$ are defined. Later, those functions are used in the computation)
# Define a function, class, and object for Theis Well analysis

def well_function(u):
    return scipy.special.exp1(u)

def theis_u(T,S,r,t):
    u = r ** 2 * S / 4. / T / t
    return u

def theis_s(Q, T, u):
    s = Q / 4. / np.pi / T * well_function(u)
    return s

# (Here, the method computes the data for the well function. Those data can be used to generate a type curve.)
u_max = 1
r_max = 10000
u  = [u_max for x in range(r_max)]
u_inv  = [r_max/u_max for x in range(r_max)]
w_u  = [well_function(u_max/r_max) for x in range(r_max)]

def compute_s(T, S, t, Q, r):
    u = theis_u(T, S, r, t)
    s = theis_s(Q, T, u)
    return s

def plot_s (r, T, S, Q, t_search, max_s, max_t):
    # Range of delta_h / delta_l values (hydraulic gradient)
    t = np.linspace(1, max_t, 200)
    
    # Compute Q for each hydraulic gradient
    s  = compute_s(T, S, t, Q, r)

    # Compute s for a specific point
    x_point = t_search
    y_point = compute_s(T, S, t_search, Q, r)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    
    plt.plot(t, s, linewidth=3., color='r', label=r'Drawdown prediction')
    plt.xlim(1, max_t)
    plt.ylim(max_s, 0)
    plt.plot(x_point,y_point, marker='o', color='b',linestyle ='None', label='drawdown output') 
    plt.xlabel(r'Time in sec', fontsize=14)
    plt.ylabel(r'Drawdown in m', fontsize=14)
    plt.title('Drawdown prediction with Theis', fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.show()
    
    print("Time since pumping start (in s): ", x_point)
    print('Drawdown at this distance (in m):  %5.2f' %y_point)
    print('Distance from the well (in ): %8.2f' %r)
    
interact(plot_s,
         r = widgets.FloatSlider(value=100,min=1, max=200, step=1,readout=True),
         T = widgets.FloatLogSlider(value=0.03,base=10,min=-6, max=4, step=0.1,readout=True,readout_format='.2e'),
         S = widgets.FloatLogSlider(value=0.003,base=10,min=-9, max=1, step=0.1,readout=True,readout_format='.2e'),
         Q = widgets.FloatSlider(value=0.005,min=0.001, max=1, step=0.001,readout=True),
         t_search = widgets.FloatSlider(value=60,min=60, max=864000, step=60,readout=True),
         max_t = widgets.FloatSlider(value=86400,min=600, max=864000, step=600,readout=True),
         max_s = widgets.FloatSlider(value=10,min=1, max=50, step=1,readout=True))

interactive(children=(FloatSlider(value=100.0, description='r', max=200.0, min=1.0, step=1.0), FloatLogSlider(…

<function __main__.plot_s(r, T, S, Q, t_search, max_s, max_t)>